In [16]:
class Vec2D:
    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y

    @staticmethod
    def zero():
        return Vec3D(0, 0, 0)

    def __add__(self, other):
        return Vec3D(self.x + other.x, self.y + other.y)

    def __iter__(self):
        yield self.x
        yield self.y

    def __repr__(self) -> str:
        return f"Vec2D({self.x}, {self.y})"


class Vec3D:
    @staticmethod
    def zero():
        return Vec3D(0, 0, 0)

    def __add__(self, other):
        return Vec3D(self.x + other.x, self.y + other.y, self.z + other.z)

    def __sub__(self, other):
        return self + -other

    def __neg__(self):
        return Vec3D(-self.x, -self.y, -self.z)

    def __repr__(self) -> str:
        return f"Vec3D({self.x}, {self.y}, {self.z})"

    def __init__(self, x: float, y: float, z: float) -> None:
        self.x = x
        self.y = y
        self.z = z

    def dot(self, other):
        return self.x * other.x + self.y * other.y + self.z * other.z

    def unit_dot(self, other):
        return self.unit().dot(other.unit())

    def unit(self):
        mag = self.magnitude()
        return Vec3D(self.x / mag, self.y / mag, self.z / mag)

    def magnitude(self):
        return (self.x**2 + self.y**2 + self.z**2)**0.5 

    def __iter__(self):
        yield self.x
        yield self.y
        yield self.z


class Rect2D:
    def __init__(self, pos: Vec2D, size: Vec2D) -> None:
        self.position = pos
        self.size = size

    @staticmethod
    def from_center_size(pos_center: Vec2D, size: Vec2D):
        position = Vec2D(pos_center.x - size.x / 2, pos_center.y - size.y / 2)
        return Rect2D(position, size)

    def left(self):
        return self.position.x

    def right(self):
        return self.position.x + self.size.x

    def top(self):
        return self.position.y

    def bottom(self):
        return self.position.y + self.size.y

    def width(self):
        return self.size.x

    def height(self):
        return self.size.y


class View:
    def __init__(
        self, position: Vec2D, canvas_rect: Rect2D, image_rect: Rect2D, near_clip: float
    ) -> None:
        self.position = position
        self.canvas_rect = canvas_rect
        self.image_rect = image_rect
        self.near_clip = near_clip


class Triangle:
    def __init__(self, color: str, *vertices) -> None:
        self.color = color
        self.verticies = list(vertices)
        self.albedo = 0.18

    def __iter__(self):
        for vertex in self.verticies:
            yield vertex

    def __repr__(self):
        return f'Triangle({", ".join(map(lambda v: str(v), self.verticies))})'
    
    def normal(self) -> Vec3D:
        u = self.verticies[1] - self.verticies[0] 
        v = self.verticies[2] - self.verticies[0] 

        n_x = (u.y * v.z) - (u.z * v.y)
        n_y = (u.z * v.x) - (u.x * v.z)
        n_z = (u.x * v.y) - (u.y * v.x)

        return Vec3D(n_x, n_y, n_z)

    def centroid(self) -> Vec3D:
        x_center = (self.verticies[0].x + self.verticies[1].x + self.verticies[2].x) / 3
        y_center = (self.verticies[0].y + self.verticies[1].y + self.verticies[2].y) / 3
        z_center = (self.verticies[0].z + self.verticies[1].z + self.verticies[2].z) / 3
        return Vec3D(x_center, y_center, z_center)

    def rasterize(self, view: View):
        assert len(list(self)) == 3, 'cant project 2d triangle'
        return Triangle(self.color,
         *[
            cam_pt_to_rast_pt(view, vertex) for vertex in self
         ][:-1])




class Shape3D:
    def __init__(self, *args) -> None:
        self.triangles = list(args)

    def __iter__(self):
        for triangle in self.triangles:
            yield triangle

class Light:
    def __init__(self, position: Vec3D, color: str, intensity: float):
        self.position = position
        self.color = color
        self.intensity = intensity

    def reflect(self, triangle: Triangle):
        triangle_normal = triangle.normal()
        facing_direction: Vec3D = triangle.centroid() - self.position
        facing_ratio = facing_direction.unit_dot(triangle_normal)
        return facing_ratio * self.intensity


import math
import drawSvg as draw

# cam_po = point in camera space
def project(view: View, cam_pt: Vec3D) -> Vec2D:
    # print(view.near_clip, cam_pt.x, cam_pt.z)
    screen_x = view.near_clip * cam_pt.x / -cam_pt.z
    screen_y = view.near_clip * cam_pt.y / -cam_pt.z

    screen_pt = Vec2D(screen_x, screen_y)

    # if (abs(screen_x) >= view.canvas_rect.width() / 2) or (
    #     abs(screen_y) >= view.canvas_rect.height() / 2
    # ):
    #     print(f"screen pt{screen_pt} is out of screen bounds")
    #     return None

    print(f"screen pt: {screen_pt}")
    return screen_pt


def map_to_ndc(view: View, screen_pt: Vec3D) -> Vec2D:
    # ndc_pt_x = (screen_pt.x + (view.canvas_rect.width() / 2)) / view.canvas_rect.width()
    # ndc_pt_y = (
    #     screen_pt.y + (view.canvas_rect.height() / 2)
    # ) / view.canvas_rect.height()
    ndc_pt_x = (screen_pt.x / view.canvas_rect.width()) + 0.5
    ndc_pt_y = (screen_pt.y / view.canvas_rect.height()) + 0.5
    ndc_pt = Vec2D(ndc_pt_x, ndc_pt_y)
    print(f"ndc pt: {ndc_pt}")
    return ndc_pt


def rasterize(view: View, ndc_pt: Vec2D, cam_pt: Vec3D) -> Vec3D:
    rast_x = math.floor(ndc_pt.x * view.image_rect.width())
    rast_y = math.floor((1 - ndc_pt.y) * view.image_rect.height())
    rast_z = -cam_pt.z

    rast_pt = Vec3D(rast_x, rast_y, rast_z)
    if rast_pt.z <= 0:
        print(f"rast pt {rast_pt} behind camera")
        return None

    print(f"rast pt: {rast_pt}\n")
    return rast_pt


def cam_pt_to_rast_pt(view: View, cam_pt: Vec2D):
    screen_pt = project(view, cam_pt)
    if screen_pt != None:
        ndc_pt = map_to_ndc(view, screen_pt)
        rast_pt = rasterize(view, ndc_pt, cam_pt)
        return rast_pt


def generate_cube(center_pos: Vec3D, color: str, size: float) -> Shape3D:
    d = size / 2  # delta
    left_top_front = center_pos + Vec3D(-d, d, d)
    left_top_back = center_pos + Vec3D(-d, d, -d)
    left_bottom_front = center_pos + Vec3D(-d, -d, d)
    left_bottom_back = center_pos + Vec3D(-d, -d, -d)

    right_top_front = center_pos + Vec3D(d, d, d)
    right_top_back = center_pos + Vec3D(d, d, -d)
    right_bottom_front = center_pos + Vec3D(d, -d, d)
    right_bottom_back = center_pos + Vec3D(d, -d, -d)

    left_lower = Triangle(color, left_top_back, left_bottom_back, left_bottom_front)
    left_upper = Triangle(color, left_top_back, left_top_front, left_bottom_front)

    right_lower = Triangle(color, right_top_back, right_bottom_back, right_bottom_front)
    right_upper = Triangle(color, right_top_back, right_top_front, right_bottom_front)

    front_lower = Triangle(color, left_bottom_front, left_top_front, right_bottom_front)
    front_upper = Triangle(color, right_top_front, left_top_front, right_bottom_front)

    back_lower = Triangle(color, left_bottom_back, left_top_back, right_bottom_back)
    back_upper = Triangle(color, right_top_back, left_top_back, right_bottom_back)

    top_lower = Triangle(color, left_top_front, left_top_back, right_top_front)
    top_upper = Triangle(color, right_top_back, left_top_back, right_top_front)

    bottom_lower = Triangle(color, left_bottom_front, left_bottom_back, right_bottom_front)
    bottom_upper = Triangle(color, right_bottom_back, left_bottom_back, right_bottom_front)

    return Shape3D(
        left_lower,
        left_upper,
        right_lower,
        right_upper,
        front_lower,
        front_upper,
        back_lower,
        back_upper,
        top_lower,
        top_upper,
        bottom_lower,
        bottom_upper,
    )


    pass

def render_point3d_to_svg(view: View, svg, point3d):
    if point3d != None:
        point2d = cam_pt_to_rast_pt(view, point3d)
        print(point2d)
        if point2d != None:
            svg.append(
                draw.Circle(
                    point2d.x, (view.image_rect.height() - point2d.y), 2, fill="white"
                )
            )


def render_triangle2d(svg, triangle2d: Triangle):
    print(triangle2d)
    print(list((v.x, v.y) for v in triangle2d))
    svg.append(
        draw.Lines(
            ((v.x, v.y) for v in triangle2d),
            close=False,
            fill=triangle2d.color,
            stroke="black",
        )
    )


def render_triangle_3d_to_2d(view: View, lights, svg, triangle3d: Triangle):
    points3d = list(triangle3d)
    # points2d = [
    #     pt_coord
    #     for pt in map(
    #         lambda pt: pt,#(pt.x, pt.y),
    #         filter(
    #             lambda pt: pt != None,
    #             map(lambda pt: cam_pt_to_rast_pt(view, pt), points3d),
    #         ),
    #     )
    #     for pt_coord in pt
    # ]
    i = 0
    for light in lights:
        i += light.reflect(triangle3d)
    i = min(max(i, 0), 1)
    triangle2d = triangle3d.rasterize(view) #Triangle(triangle3d.color, points2d)
    # print('t', triangle2d)
    render_triangle2d(svg, triangle2d)


def render_shape3d_to_svg(view: View, lights, svg, shape: Shape3D):
    for triangle in shape:
        print(triangle)
        render_triangle_3d_to_2d(view, lights, svg, triangle)

from ast import literal_eval

def str_to_hex(str):
    return literal_eval(f'0x{str}')

def scale_color(color_str, scale):
    def s(i):
        return hex(int(str_to_hex(color_str[i]) * scale))[2]
    return f'#{s(1)}{s(2)}{s(3)}'

def main():
    view_pos = Vec3D.zero()
    view_canvas_rect = Rect2D.from_center_size(Vec2D.zero(), Vec2D(10, 10))
    view_image_rect = Rect2D(Vec2D.zero(), Vec2D(512, 384))

    view = View(view_pos, view_canvas_rect, view_image_rect, 1)
    point_3d = Vec3D(20, 23, -2)
    # point_2d = cam_pt_to_rast_pt(view, point_3d)

    t = Triangle(
        '#f0f',
        Vec3D(1, 1, -2),
        Vec3D(1, -1.5, -2),
        Vec3D(-1, -1, -2),
    )

    t2 = Triangle('#f0f',
        Vec3D(-3, -3, -4),
        Vec3D(-4, 3, -2),
        Vec3D(1, 4, -4),
    )

    svg = draw.Drawing(
        view.image_rect.width(),
        view.image_rect.height(),
        origin=(0, 0),
        displayInline=False,
    )

    bg = draw.Rectangle(
        view_image_rect.position.x,
        view_image_rect.position.y,
        view_image_rect.width(),
        view_image_rect.height(),
        fill="#000000",
    )
    svg.append(bg)

    light = Light(Vec3D(0, 10, 0), '#fff', 1)
    lights = [light]

    cube = generate_cube(Vec3D(90, 150, -100), '#f0f', 203)
    render_shape3d_to_svg(view, lights, svg, cube)


    # render_triangle_3d_to_2d(view, lights, svg, t)
    # render_triangle_3d_to_2d(view, lights, svg, t2)

    # print('---------------------------')


    svg.rasterize()
    return svg


# run in a jupyter notebook
main()


Triangle(Vec3D(-11.5, 251.5, -201.5), Vec3D(-11.5, 48.5, -201.5), Vec3D(-11.5, 48.5, 1.5))
screen pt: Vec2D(-0.05707196029776675, 1.248138957816377)
ndc pt: Vec2D(0.49429280397022335, 0.6248138957816377)
rast pt: Vec3D(253, 144, 201.5)

screen pt: Vec2D(-0.05707196029776675, 0.24069478908188585)
ndc pt: Vec2D(0.49429280397022335, 0.5240694789081886)
rast pt: Vec3D(253, 182, 201.5)

screen pt: Vec2D(7.666666666666667, -32.333333333333336)
ndc pt: Vec2D(1.2666666666666666, -2.7333333333333334)
rast pt Vec3D(648, 1433, -1.5) behind camera
Triangle(Vec3D(253, 144, 201.5), Vec3D(253, 182, 201.5))
[(253, 144), (253, 182)]


TypeError: __init__() missing 1 required positional argument: 'sy'

In [36]:
scale_color('#fff', 0.5)

'#777'

In [ ]:
#777